<a href="https://colab.research.google.com/github/cristianbossolasco/scraper-agencias-viajes/blob/main/scraper_busplus_BeautifulSoup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta
import re
import warnings
warnings.filterwarnings("ignore")
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
url = 'https://raw.githubusercontent.com/cristianbossolasco/scraper-agencias-viajes/main/busplus_files/paradas.csv'
df_paradas = pd.read_csv(url, delimiter = ';', encoding = 'unicode_escape')
df_paradas.nombre = df_paradas.nombre.apply(lambda x: x.encode('iso-8859-1').decode('utf-8'))
df_paradas.head()

,id,direccion,latitud,longitud,nombre
0,6,,-35137317600,-6.045238e+10,Bragado - Caminera (Buenos Aires - ARG)
1,7,,-35838385600,-6.187472e+10,Pehuajo - ESSO (Buenos Aires - ARG)
2,8,,-35636855700,-6.135356e+10,Carlos Casares - Shell (Buenos Aires - ARG)
3,10,,-34905308000,-5.795551e+10,La Plata - Terminal (Buenos Aires - ARG)
4,11,,-35451965600,-6.088127e+10,9 de Julio - Terminal (Buenos Aires - ARG)


In [22]:
def get_data_from_page(origen_parada, destino_parada, fecha_ida):

  cookies = {
      '_gcl_au': '1.1.941593195.1676950807',
      '_ga': 'GA1.3.658349179.1676950808',
      '__kdtv': 't%3D1676950808075%3Bi%3D2c1e3b9f3f5261097e6794daddb5fd390f3a2759',
      '_kdt': '%7B%22t%22%3A1676950808075%2C%22i%22%3A%222c1e3b9f3f5261097e6794daddb5fd390f3a2759%22%7D',
      '_gid': 'GA1.3.162808660.1677382760',
      'ci_session': 'a%3A11%3A%7Bs%3A10%3A%22session_id%22%3Bs%3A32%3A%2221adf16643db557f91caef6f7f9c68b6%22%3Bs%3A10%3A%22ip_address%22%3Bs%3A13%3A%22192.168.1.165%22%3Bs%3A10%3A%22user_agent%22%3Bs%3A111%3A%22Mozilla%2F5.0+%28Windows+NT+10.0%3B+Win64%3B+x64%29+AppleWebKit%2F537.36+%28KHTML%2C+like+Gecko%29+Chrome%2F110.0.0.0+Safari%2F537.36%22%3Bs%3A13%3A%22last_activity%22%3Bi%3A1677445278%3Bs%3A9%3A%22user_data%22%3Bs%3A0%3A%22%22%3Bs%3A5%3A%22login%22%3Bb%3A1%3Bs%3A7%3A%22usuario%22%3Bs%3A11%3A%22Busplus.com%22%3Bs%3A7%3A%22agencia%22%3Bs%3A11%3A%22WEB+BusPlus%22%3Bs%3A3%3A%22key%22%3Bs%3A36%3A%2280C2F22D-0119-4E65-AB52-3B55724EB91E%22%3Bs%3A4%3A%22logo%22%3Bs%3A11%3A%22busplus.png%22%3Bs%3A7%3A%22favicon%22%3Bs%3A11%3A%22busplus.ico%22%3B%7D523804e12543e07963f689fc7e3a45e940479eea',
  }
  headers = {
      'authority': 'checkout.busplus.com.ar',
      'accept': 'text/plain, */*; q=0.01',
      'accept-language': 'es-ES,es;q=0.9,en;q=0.8',
      'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
      # 'cookie': '_gcl_au=1.1.941593195.1676950807; _ga=GA1.3.658349179.1676950808; __kdtv=t%3D1676950808075%3Bi%3D2c1e3b9f3f5261097e6794daddb5fd390f3a2759; _kdt=%7B%22t%22%3A1676950808075%2C%22i%22%3A%222c1e3b9f3f5261097e6794daddb5fd390f3a2759%22%7D; _gid=GA1.3.162808660.1677382760; ci_session=a%3A11%3A%7Bs%3A10%3A%22session_id%22%3Bs%3A32%3A%2221adf16643db557f91caef6f7f9c68b6%22%3Bs%3A10%3A%22ip_address%22%3Bs%3A13%3A%22192.168.1.165%22%3Bs%3A10%3A%22user_agent%22%3Bs%3A111%3A%22Mozilla%2F5.0+%28Windows+NT+10.0%3B+Win64%3B+x64%29+AppleWebKit%2F537.36+%28KHTML%2C+like+Gecko%29+Chrome%2F110.0.0.0+Safari%2F537.36%22%3Bs%3A13%3A%22last_activity%22%3Bi%3A1677445278%3Bs%3A9%3A%22user_data%22%3Bs%3A0%3A%22%22%3Bs%3A5%3A%22login%22%3Bb%3A1%3Bs%3A7%3A%22usuario%22%3Bs%3A11%3A%22Busplus.com%22%3Bs%3A7%3A%22agencia%22%3Bs%3A11%3A%22WEB+BusPlus%22%3Bs%3A3%3A%22key%22%3Bs%3A36%3A%2280C2F22D-0119-4E65-AB52-3B55724EB91E%22%3Bs%3A4%3A%22logo%22%3Bs%3A11%3A%22busplus.png%22%3Bs%3A7%3A%22favicon%22%3Bs%3A11%3A%22busplus.ico%22%3B%7D523804e12543e07963f689fc7e3a45e940479eea',
      'origin': 'https://checkout.busplus.com.ar',
      'referer': 'https://checkout.busplus.com.ar/servicios',
      'sec-ch-ua': '"Chromium";v="110", "Not A(Brand";v="24", "Google Chrome";v="110"',
      'sec-ch-ua-mobile': '?0',
      'sec-ch-ua-platform': '"Windows"',
      'sec-fetch-dest': 'empty',
      'sec-fetch-mode': 'cors',
      'sec-fetch-site': 'same-origin',
      'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
      'x-requested-with': 'XMLHttpRequest',
  }

  data = {
      'combinaciones_ida': '0',
      'combinaciones_vuelta': '0',
      'origen_parada': origen_parada,
      'destino_parada': destino_parada,
      'fecha_ida': fecha_ida,
      'fecha_ida_txt': '',
      'fechaFiltroIda': '0',
      'lugarPrecioIda': '0',
      'fecha_vuelta': '',
      'fecha_vuelta_txt': '',
      'fechaFiltroVuelta': '0',
      'lugarPrecioVuelta': '0',
      'solPersona': '1',
      'cupondescuento': '',
  }

  response = requests.post('https://checkout.busplus.com.ar/servicios/obtenerServicios', cookies=cookies, headers=headers, data=data)

  s = BeautifulSoup(response.content, 'html.parser') 
  textarea = s.find_all('textarea')[0]
  return json.loads(textarea.text)

def get_data_from_item(json_item):

  ls_categorias = []
  for categoria in json_item['categorias']:

    bEncontro = False
    for i in range(len(ls_categorias)):
      if ls_categorias[i]['descripcion_categoria'] == categoria['descripcion_categoria']:
        bEncontro = True
        break

    if bEncontro:
      ls_categorias[i]['butacas_libres'] += categoria['butacas_libres']
    else:

      json_categoria = {
        'descripcion_categoria': categoria['descripcion_categoria'],
        'importe': categoria['importe'],
        'butacas_libres': categoria['butacas_libres']
      }
      ls_categorias.append(json_categoria)

  if json_item['categorias'] == 1: # Si es un viaje directo

    item_json = {
      "origen_parada": json_item['idorigen'],
      "nombre_origen_parada": json_item['descripcion_origen'],
      "destino_parada": json_item['iddestino'],
      "nombre_destino_parada": json_item['descripcion_destino'],
      "fecha_ida": fecha_ida,
      "empresa": json_item['descripcion_web'],
      "salida": json_item['fecha_sale'],
      "llegada": json_item['fecha_llega'],
      "escala": "directo",
      "duracion": json_item['duracion'],
      "categorias": ls_categorias
    }

  else: # Es un viaje con escalas

    item_json = {
      "origen_parada": json_item['idorigen'],
      "nombre_origen_parada": json_item['descripcion_origen'],
      "destino_parada": json_item['iddestino'],
      "nombre_destino_parada": json_item['descripcion_destino'],
      "fecha_ida": fecha_ida,
      "empresa": json_item['descripcion_web'],
      "salida": json_item['fecha_sale'],
      "llegada": json_item['fecha_llega'],
      "escala": "escala",
      "duracion": json_item['duracion'],
      "combinacion": {
        "idcombinacion": json_item['idcombinacion'],
        "combinacion_orden": json_item['combinacion_orden'],
        "combinacion_espera": json_item['combinacion_espera'],
        "combinacion_duracion": json_item['combinacion_duracion'],
      },
      "categorias": ls_categorias
    }


  return item_json

def get_data_directo(json_item):
  ls_categorias = []
  for categoria in json_item['categorias']:

    bEncontro = False
    for i in range(len(ls_categorias)):
      if ls_categorias[i]['descripcion_categoria'] == categoria['descripcion_categoria']:
        bEncontro = True
        break

    if bEncontro:
      ls_categorias[i]['butacas_libres'] += categoria['butacas_libres']
    else:

      json_categoria = {
        'descripcion_categoria': categoria['descripcion_categoria'],
        'importe': categoria['importe'],
        'butacas_libres': categoria['butacas_libres']
      }
      ls_categorias.append(json_categoria)

  item_json = {
    "origen_parada": json_item['idorigen'],
    "nombre_origen_parada": json_item['descripcion_origen'],
    "destino_parada": json_item['iddestino'],
    "nombre_destino_parada": json_item['descripcion_destino'],
    "fecha_ida": fecha_ida,
    "empresa": json_item['descripcion_web'],
    "salida": json_item['fecha_sale'],
    "llegada": json_item['fecha_llega'],
    "escala": 'directo',
    "duracion": json_item['duracion'],
    "categorias": ls_categorias
  }

  return item_json  


In [45]:
def get_data(origen_parada, destino_parada, fecha_ida):

  json_response = get_data_from_page(origen_parada, destino_parada, fecha_ida)
  
  ls_directo = []
  ls_escala = []
  for i in range(len(json_response)):
    item = json_response[i]
    print(item['combinacion_orden'])
    if item['combinacion_orden'] == None:
      ls_directo.append(get_data_directo(json_response[i]))
    else:

      item_escala = get_data_from_item(json_response[i])

      if len(ls_escala) == 0:
        ls_escala.append(item_escala)
      else:

        escalas = [{},{},{},{}]

        bEncontro = False
        for i in range(len(ls_escala)):
          if ls_escala[i]['combinacion']['idcombinacion'] == item_escala['combinacion']['idcombinacion']:
            combinacion_orden = ls_escala[i]['combinacion']['combinacion_orden']
            escalas[combinacion_orden - 1] = ls_escala[i]
            bEncontro = True
            break

          if bEncontro:
            combinacion_orden = item_escala['combinacion']['combinacion_orden']
            escalas[combinacion_orden - 1] = item_escala

            categorias = []
            for cat_tramo_1 in escalas[0]["categorias"]:
              for cat_tramo_2 in escalas[1]["categorias"]:
                
                if cat_tramo_1['descripcion_categoria'] == cat_tramo_2['descripcion_categoria']:
                  descripcion_categoria = cat_tramo_1['descripcion_categoria']
                else:
                  descripcion_categoria = cat_tramo_1['descripcion_categoria'] + " | " + cat_tramo_2['descripcion_categoria']

                importe1 = float(cat_tramo_1['importe'])
                importe2 = float(cat_tramo_2['importe'])

                if cat_tramo_1['butacas_libres'] <= cat_tramo_2['butacas_libres']:
                  butacas_libres = cat_tramo_1['butacas_libres']
                else:
                  butacas_libres = cat_tramo_2['butacas_libres']

                json_categoria = {
                  'descripcion_categoria': descripcion_categoria,
                  'importe': importe1 + importe2,
                  'butacas_libres': butacas_libres
                }
                categorias.append(json_categoria)

            item_json = {
              "origen_parada": escalas[0]['idorigen'],
              "nombre_origen_parada": escalas[0]['descripcion_origen'],
              "destino_parada": escalas[1]['iddestino'],
              "nombre_destino_parada": escalas[1]['descripcion_destino'],
              "fecha_ida": fecha_ida,
              "empresa": escalas[0]['descripcion_web'] if escalas[0]['descripcion_web'] == escalas[1]['descripcion_web'] else escalas[0]['descripcion_web'] + ' | ' + escalas[1]['descripcion_web'],
              "salida": escalas[0]['fecha_sale'],
              "llegada": escalas[1]['fecha_llega'],
              "escala": "escala",
              "duracion": escalas[0]['combinacion']['combinacion_duracion'],
              "categorias": categorias
            }



          else:

            ls_escala.append(item_escala)

  return ls_directo

def scrapear_viajes(ls_origenes, ls_destinos, n_dias):
  """
    Busca todas las combinaciones entre los origenes y el destinos, y scrapena n_dias hacia adelante a partir de mañana

    Args:
      ls_origenes (list): lista con los id de origen
      ls_destinos (list): lista con los id de destino
      n_dias (list): cantidad de dias que se va a escrapear a partir de mañana

    Returns:
      lista de json con los viajes
  """

  resultado = []

  for iOrigen in range(len(ls_origenes)):

    id_origen = ls_origenes[iOrigen]
  
    for iDestino in range(len(ls_destinos)):

      id_destino = ls_destinos[iDestino]
      if id_destino == id_origen: continue
  
      # incializo con la fecha de mañana
      date = datetime.now() + timedelta(1) 
      for j in range(n_dias): 

          year = str(date.year)
          month = ('00' + str(date.month))[-2:]
          day = ('00' + str(date.day))[-2:]
          fecha = f'{day}/{month}/{year}'
          
          result = get_data(id_origen, id_destino, fecha)

          resultado.extend(result)
          date += timedelta(days=1)

  return resultado

In [46]:
ls_origenes = [28]
ls_destinos  = [166]

result = scrapear_viajes(ls_origenes, ls_destinos, 1)
df_results = pd.DataFrame.from_records(result)
df_results

None
None
None
None
None
None
None


,origen_parada,nombre_origen_parada,destino_parada,nombre_destino_parada,fecha_ida,empresa,salida,llegada,escala,duracion,categorias
0,28,(BUE) Retiro - Terminal,166,Rosario - Terminal,05/03/2023,Rutamar - Via Tac,2023-02-27 08:30:00.000,2023-02-27 12:30:00.000,directo,04:00,"[{'descripcion_categoria': 'Cama Ejecutivo', '..."
1,28,(BUE) Retiro - Terminal,166,Rosario - Terminal,05/03/2023,Rutamar - Via Tac,2023-02-27 10:30:00.000,2023-02-27 14:30:00.000,directo,04:00,"[{'descripcion_categoria': 'Cama Ejecutivo', '..."
2,28,(BUE) Retiro - Terminal,166,Rosario - Terminal,05/03/2023,Via Bariloche,2023-02-27 11:15:00.000,2023-02-27 15:40:00.000,directo,04:25,"[{'descripcion_categoria': 'Estandar', 'import..."
3,28,(BUE) Retiro - Terminal,166,Rosario - Terminal,05/03/2023,Rutamar - Via Tac,2023-02-27 12:30:00.000,2023-02-27 16:30:00.000,directo,04:00,"[{'descripcion_categoria': 'Cama Ejecutivo', '..."
4,28,(BUE) Retiro - Terminal,166,Rosario - Terminal,05/03/2023,Rutamar - Via Tac,2023-02-27 14:30:00.000,2023-02-27 18:30:00.000,directo,04:00,"[{'descripcion_categoria': 'Cama Ejecutivo', '..."
5,28,(BUE) Retiro - Terminal,166,Rosario - Terminal,05/03/2023,Via Bariloche,2023-02-27 16:20:00.000,2023-02-27 21:00:00.000,directo,04:40,"[{'descripcion_categoria': 'Cama Ejecutivo', '..."
6,28,(BUE) Retiro - Terminal,166,Rosario - Terminal,05/03/2023,Rutamar - Via Tac,2023-02-27 19:30:00.000,2023-02-27 23:30:00.000,directo,04:00,"[{'descripcion_categoria': 'Cama Ejecutivo', '..."


In [47]:
result

[{'origen_parada': 28,
  'nombre_origen_parada': '(BUE) Retiro - Terminal ',
  'destino_parada': 166,
  'nombre_destino_parada': 'Rosario - Terminal',
  'fecha_ida': '05/03/2023',
  'empresa': 'Rutamar - Via Tac',
  'salida': '2023-02-27 08:30:00.000',
  'llegada': '2023-02-27 12:30:00.000',
  'escala': 'directo',
  'duracion': '04:00',
  'categorias': [{'descripcion_categoria': 'Cama Ejecutivo',
    'importe': '4200.00',
    'butacas_libres': 19}]},
 {'origen_parada': 28,
  'nombre_origen_parada': '(BUE) Retiro - Terminal ',
  'destino_parada': 166,
  'nombre_destino_parada': 'Rosario - Terminal',
  'fecha_ida': '05/03/2023',
  'empresa': 'Rutamar - Via Tac',
  'salida': '2023-02-27 10:30:00.000',
  'llegada': '2023-02-27 14:30:00.000',
  'escala': 'directo',
  'duracion': '04:00',
  'categorias': [{'descripcion_categoria': 'Cama Ejecutivo',
    'importe': '4200.00',
    'butacas_libres': 32}]},
 {'origen_parada': 28,
  'nombre_origen_parada': '(BUE) Retiro - Terminal ',
  'destino_pa

In [ ]:
result

In [ ]:
id_origen = 28
id_destino = 166
fecha_ida = '05/03/2023'

json_response = get_data_from_page(id_origen, id_destino, fecha_ida)
json_response

In [ ]:
ls_directo = []
ls_escala = []
for i in range(len(json_response)):
  item = json_response[i]
  if item['categorias'] == 1:
    ls_directo.append(get_data_from_item(json_response[i]))
  else:

    item_escala = get_data_from_item(json_response[i])

    if len(ls_escala) == 0:
      ls_escala.append(item_escala)
    else:

      escalas = [{},{},{},{}]

      bEncontro = False
      for i in range(len(ls_escala)):
        if ls_escala[i]['combinacion']['idcombinacion'] == item_escala['combinacion']['idcombinacion']:
          combinacion_orden = ls_escala[i]['combinacion']['combinacion_orden']
          escalas[combinacion_orden - 1] = ls_escala[i]
          bEncontro = True
          break

        if bEncontro:
          combinacion_orden = item_escala['combinacion']['combinacion_orden']
          escalas[combinacion_orden - 1] = item_escala

          categorias = []
          for cat_tramo_1 in escalas[0]["categorias"]:
            for cat_tramo_2 in escalas[1]["categorias"]:
              
              if cat_tramo_1['descripcion_categoria'] == cat_tramo_2['descripcion_categoria']:
                descripcion_categoria = cat_tramo_1['descripcion_categoria']
              else:
                descripcion_categoria = cat_tramo_1['descripcion_categoria'] + " | " + cat_tramo_2['descripcion_categoria']

              importe1 = float(cat_tramo_1['importe'])
              importe2 = float(cat_tramo_2['importe'])

              if cat_tramo_1['butacas_libres'] <= cat_tramo_2['butacas_libres']:
                butacas_libres = cat_tramo_1['butacas_libres']
              else:
                butacas_libres = cat_tramo_2['butacas_libres']

              json_categoria = {
                'descripcion_categoria': descripcion_categoria,
                'importe': importe1 + importe2,
                'butacas_libres': butacas_libres
              }
              categorias.append(json_categoria)

          item_json = {
            "origen_parada": escalas[0]['idorigen'],
            "nombre_origen_parada": escalas[0]['descripcion_origen'],
            "destino_parada": escalas[1]['iddestino'],
            "nombre_destino_parada": escalas[1]['descripcion_destino'],
            "fecha_ida": fecha_ida,
            "empresa": escalas[0]['descripcion_web'] if escalas[0]['descripcion_web'] == escalas[1]['descripcion_web'] else escalas[0]['descripcion_web'] + ' | ' + escalas[1]['descripcion_web'],
            "salida": escalas[0]['fecha_sale'],
            "llegada": escalas[1]['fecha_llega'],
            "escala": "escala",
            "duracion": escalas[0]['combinacion']['combinacion_duracion'],
            "categorias": categorias
          }



        else:

          ls_escala.append(item_escala)


ls_escala

In [ ]:
for item in ls_escala:
  print(item)